#Augmentation/Data-prep notebook — Summary & Requirements

## What it does
1. Convert raw German Traffic Sign Detection Benchmark (GTSDB - FullIJCNN2013) Dataset to YOLO format.  
2. Create augmented images by pasting sign crops from German Traffic Sign Recognition Benchmark (GTSRB) Dataset onto empty background frames using polygon placement.  
3. Experiments used ~102 empty background frames (these are **NOT** included in the repo), but the notebook contains a cell/script `extract_empty_backgrounds` to recreate empties from your local train images.

## Inputs (required to run)
- Download GTSRB ([you can download it from here](https://www.kaggle.com/datasets/meowmeowmeowmeowmeow/gtsrb-german-traffic-sign)) and use `GTSRB/Train/` and `GTSRB/Train.csv`.  
- `data/images/empty_bg_images/` — background images (NOT included). Use the `extract_empty_backgrounds` cell to create these from your training images.  
- `data/annotations/allowed_areas.json` — COCO-style polygons (this JSON **is included** in the [repo](https://github.com/WahburRehman/traffic-sign-and-speed-violation-detector)).

## Outputs
- `data/images_all/`, `data/labels_all/` (YOLO converted)  
- `data/images/aug_images_polygon/`, `data/labels/aug_labels_polygon/` (augmented outputs)  
- Optional: `data/images/train/`, `data/images/val/`, `data/labels/train/`, `data/labels/val/` after splitting/assembly

## How to run (high level)
1. Provide required inputs (see above) or run in Colab and mount Drive.  
2. Run cells in order: setup → download/extract → convert `gt.txt` → run `extract_empty_backgrounds` (if needed) → augmentation.  
3. After augmentation, assemble `train` by combining original and `aug_images_polygon/` or update `data.yaml` accordingly.

## Notes: PATH CONFIGURATION - UPDATE THESE FOR YOUR ENVIRONMENT:
- PROJ: Your project root directory
- GDRIVE_ROOT: Your Google Drive mount point (if using Colab)
- Set paths for `GTSRB/Train/` and `GTSRB/Train.csv` accordingly.
- Make sure to use the correct path for `allowed_areas.json`.

In [ ]:
import os, shutil, zipfile, glob, cv2, json, pathlib, random,  numpy as np, pandas as pd
from pathlib import Path
pathlib.PosixPath = pathlib.Path
random.seed(42)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

PROJ = "/content/drive/MyDrive/traffic-sign-violation-mvp"
RAW  = f"{PROJ}/raw"
DATA = f"{PROJ}/data"
os.makedirs(RAW, exist_ok=True)
os.makedirs(DATA, exist_ok=True)
print("Project root:", PROJ)


In [ ]:
# Download GTSDB (FullIJCNN2013)
!mkdir -p {RAW}
!wget -q -O {RAW}/FullIJCNN2013.zip https://sid.erda.dk/public/archives/ff17dc924eba88d5d01a807357d6614c/FullIJCNN2013.zip

# Extract
zip_path = f"{RAW}/FullIJCNN2013.zip"
with zipfile.ZipFile(zip_path, 'r') as z:
    z.extractall(f"{RAW}/FullIJCNN2013")

# Flatten inner folder if needed
%cd {RAW}
if os.path.exists("FullIJCNN2013/FullIJCNN2013"):
    !mv FullIJCNN2013/FullIJCNN2013/* FullIJCNN2013/
    !rmdir FullIJCNN2013/FullIJCNN2013

!ls FullIJCNN2013 | head


In [ ]:
IMG_DIR = f"{RAW}/FullIJCNN2013"
GT_FILE = f"{IMG_DIR}/gt.txt"

OUT_ALL_IMG = f"{DATA}/images_all"
OUT_ALL_LAB = f"{DATA}/labels_all"
os.makedirs(OUT_ALL_IMG, exist_ok=True)
os.makedirs(OUT_ALL_LAB, exist_ok=True)

ann = {}  # stem -> list of yolo lines
with open(GT_FILE, "r") as f:
    for line in f:
        parts = line.strip().split(";")
        if len(parts) != 6:
            continue
        fname, x1, y1, x2, y2, cls = parts
        x1, y1, x2, y2, cls = map(int, [x1, y1, x2, y2, cls])

        img_path = os.path.join(IMG_DIR, fname)
        img = cv2.imread(img_path)
        if img is None:
            continue
        H, W = img.shape[:2]
        x_c = (x1 + x2) / 2.0 / W
        y_c = (y1 + y2) / 2.0 / H
        bw  = (x2 - x1) / W
        bh  = (y2 - y1) / H

        stem = Path(fname).stem
        ann.setdefault(stem, []).append(f"{cls} {x_c:.6f} {y_c:.6f} {bw:.6f} {bh:.6f}")

ppm_files = sorted(glob.glob(os.path.join(IMG_DIR, "*.ppm")))
n_img, n_lab = 0, 0
for ppm in ppm_files:
    img = cv2.imread(ppm)
    if img is None:
        continue
    stem = Path(ppm).stem
    out_img = os.path.join(OUT_ALL_IMG, f"{stem}.jpg")
    if not os.path.exists(out_img):
        cv2.imwrite(out_img, img); n_img += 1

    lines = ann.get(stem, [])
    with open(os.path.join(OUT_ALL_LAB, f"{stem}.txt"), "w") as lf:
            lf.write("\n".join(lines))
    n_lab += 1

print(f"Images written: {n_img} | Label files: {n_lab} (some labels may be empty if no sign)")


In [ ]:
ALL_IMG = f"{PROJ}/data/images_all"
ALL_LAB = f"{PROJ}/data/labels_all"

TRAIN_IMG = f"{PROJ}/data/images/train"
VAL_IMG   = f"{PROJ}/data/images/val"
TRAIN_LAB = f"{PROJ}/data/labels/train"
VAL_LAB   = f"{PROJ}/data/labels/val"
for d in [TRAIN_IMG, VAL_IMG, TRAIN_LAB, VAL_LAB]:
    os.makedirs(d, exist_ok=True)

# All stems
stems = [Path(p).stem for p in glob.glob(f"{ALL_IMG}/*.jpg")]
random.Random(42).shuffle(stems)

# Pick 600 train + 300 val
train_stems = stems[:600]
val_stems   = stems[600:900]

def cp(stem, src_img, src_lab, dst_img, dst_lab):
    si, sl = f"{src_img}/{stem}.jpg", f"{src_lab}/{stem}.txt"
    if os.path.exists(si): shutil.copy2(si, dst_img)
    if os.path.exists(sl): shutil.copy2(sl, dst_lab)

for s in train_stems:
    cp(s, ALL_IMG, ALL_LAB, TRAIN_IMG, TRAIN_LAB)
for s in val_stems:
    cp(s, ALL_IMG, ALL_LAB, VAL_IMG, VAL_LAB)

print("Train imgs:", len(glob.glob(TRAIN_IMG+"/*.jpg")))
print("Val   imgs:", len(glob.glob(VAL_IMG+"/*.jpg")))


In [ ]:
# extract_empty_backgrounds

TRAIN_IMG_DIR = f"{PROJ}/data/images/train"
TRAIN_LAB_DIR = f"{PROJ}/data/labels/train"
OUT_DIR = f"{PROJ}/data/images/empty_bg_images"

os.makedirs(OUT_DIR, exist_ok=True)

jpg_paths = sorted(glob.glob(os.path.join(TRAIN_IMG_DIR, "*.jpg")))
empty_count = 0
copied = 0

for p in jpg_paths:
    stem = Path(p).stem
    lab = os.path.join(TRAIN_LAB_DIR, f"{stem}.txt")

    # consider empty if label missing OR file exists but size == 0
    is_empty = (not os.path.exists(lab)) or (os.path.exists(lab) and os.path.getsize(lab) == 0)

    if is_empty:
        dst = os.path.join(OUT_DIR, Path(p).name)
        if not os.path.exists(dst):
            shutil.copy2(p, dst)   # use shutil.move(p, dst) if you want to move instead
            copied += 1
        empty_count += 1

print(f"Total train images checked: {len(jpg_paths)}")
print(f"Images with empty/missing labels: {empty_count}")
print(f"Images copied to '{OUT_DIR}': {copied}")


In [ ]:
GDRIVE_ROOT = "/content/drive/MyDrive"

# COCO-style JSON with per-image allowed polygons/bboxes
ALLOWED_JSON = f"{GDRIVE_ROOT}/traffic-sign-violation-mvp/allowed_areas.json"
BG_BASE_DIR = f"{GDRIVE_ROOT}/traffic-sign-violation-mvp/data/images/empty_bg_images"
GTSRB_TRAIN_DIR = f"{GDRIVE_ROOT}/GTSRB/Train"
GTSRB_TRAIN_CSV = f"{GDRIVE_ROOT}/GTSRB/Train.csv"

# Output directories
OUT_IMG_DIR = f"{GDRIVE_ROOT}/traffic-sign-violation-mvp/data/images/aug_images_polygon"
OUT_LBL_DIR = f"{GDRIVE_ROOT}/traffic-sign-violation-mvp/data/labels/aug_labels_polygon"
Path(OUT_IMG_DIR).mkdir(parents=True, exist_ok=True)
Path(OUT_LBL_DIR).mkdir(parents=True, exist_ok=True)

# Augmentation parameters
TARGET_PER_CLASS = 306
MAX_SIGN_REUSE = 5

CLASSES_TO_GENERATE = [0]  # Classe 0
# CLASSES_TO_GENERATE = list(range(1, 11))  # Classes 1-10
# CLASSES_TO_GENERATE = list(range(11, 21))  # Classes 11-20
# CLASSES_TO_GENERATE = list(range(21, 31))  # Classes 21-30
# CLASSES_TO_GENERATE = list(range(31, 43))  # Classes 31-42

ALL_CLASSES = list(range(43))
random.seed(42)

# --- Load metadata (NOT images) ---
print("Loading metadata...")

# Load allowed areas JSON
with open(ALLOWED_JSON, 'r') as f:
    allowed_data = json.load(f)

# Create mapping from image_id to allowed areas
image_areas = {}
for ann in allowed_data['annotations']:
    image_id = ann['image_id']
    if image_id not in image_areas:
        image_areas[image_id] = []
    image_areas[image_id].append(ann)

# Create mapping from filename to image data
image_info = {img['file_name']: img for img in allowed_data['images']}

# Load GTSRB CSV for ROI information
df = pd.read_csv(GTSRB_TRAIN_CSV)
roi_map = {}
for _, row in df.iterrows():
    roi_map[row['Path']] = (row['Roi.X1'], row['Roi.Y1'], row['Roi.X2'], row['Roi.Y2'])

# Get all background images and sort them
bg_images = sorted([f for f in os.listdir(BG_BASE_DIR) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp'))])
print(f"Found {len(bg_images)} background images")

# Get all sign image PATHS
sign_paths_by_class = {}
for cls in ALL_CLASSES:
    cls_dir = Path(GTSRB_TRAIN_DIR) / str(cls)
    sign_paths = []
    for ext in ['*.png', '*.jpg', '*.jpeg', '*.bmp', '*.ppm']:
        sign_paths.extend([str(p) for p in cls_dir.glob(ext)])
    sign_paths_by_class[cls] = sign_paths
    print(f"Class {cls}: {len(sign_paths)} signs")

# --- Helper functions ---
def get_sign_roi(sign_path):
    """Extract ROI from sign image using CSV data"""
    # Get relative path from GTSRB root
    rel_path = str(Path(sign_path).relative_to(Path(GTSRB_TRAIN_DIR).parent))

    if rel_path in roi_map:
        return roi_map[rel_path]

    # Try with forward slashes for consistency
    rel_path_forward = rel_path.replace('\\', '/')
    if rel_path_forward in roi_map:
        return roi_map[rel_path_forward]

    return None

def load_and_process_sign(sign_path):
    """Load and process a sign image ON DEMAND"""
    img = cv2.imread(sign_path)
    if img is None:
        return None

    # Crop to ROI coordinates
    roi = get_sign_roi(sign_path)
    if roi:
        x1, y1, x2, y2 = roi
        x1, y1 = max(0, x1), max(0, y1)
        x2, y2 = min(img.shape[1], x2), min(img.shape[0], y2)
        if x2 > x1 and y2 > y1:
            img = img[y1:y2, x1:x2]

    return img


def get_polygon_center(polygon):
    """Calculate center point of a polygon"""
    points = np.array(polygon).reshape(-1, 2)
    center = np.mean(points, axis=0)
    return int(center[0]), int(center[1])

def resize_to_fit_bbox(sign_img, bbox, max_scale=0.8):
    """Resize sign to fit within bbox - NO cosmetic alterations"""
    bbox_w = bbox[2]
    bbox_h = bbox[3]

    sign_h, sign_w = sign_img.shape[:2]

    # Calculate scale factors
    scale_w = (bbox_w * max_scale) / sign_w
    scale_h = (bbox_h * max_scale) / sign_h
    scale = min(scale_w, scale_h)

    # Small random variation for natural look
    scale *= random.uniform(0.8, 0.95)

    new_w = max(10, int(sign_w * scale))
    new_h = max(10, int(sign_h * scale))

    return cv2.resize(sign_img, (new_w, new_h), interpolation=cv2.INTER_LANCZOS4)

def create_soft_mask(sign_img):
    """Create soft mask for blending"""
    if len(sign_img.shape) == 3:
        gray = cv2.cvtColor(sign_img, cv2.COLOR_BGR2GRAY)
    else:
        gray = sign_img

    # Simple thresholding to remove white backgrounds
    _, mask = cv2.threshold(gray, 220, 255, cv2.THRESH_BINARY_INV)

    # Soft edges for natural blending
    mask = cv2.GaussianBlur(mask, (3, 3), 0.5)
    return mask

def alpha_blend(bg, sign, mask, center_x, center_y):
    """Blend sign onto background"""
    h, w = sign.shape[:2]
    x1 = max(0, center_x - w // 2)
    y1 = max(0, center_y - h // 2)
    x2 = min(bg.shape[1], x1 + w)
    y2 = min(bg.shape[0], y1 + h)

    # Adjust sign and mask to fit
    sign = sign[0:y2-y1, 0:x2-x1]
    mask = mask[0:y2-y1, 0:x2-x1]

    if sign.size == 0:
        return bg, (0, 0, 0, 0)

    # Convert mask to 3 channels
    if len(mask.shape) == 2:
        mask_3ch = cv2.merge([mask, mask, mask])
    else:
        mask_3ch = mask

    mask_float = mask_3ch.astype(float) / 255.0

    # Blend
    bg_roi = bg[y1:y2, x1:x2]
    blended = bg_roi.astype(float) * (1 - mask_float) + sign.astype(float) * mask_float
    blended = np.clip(blended, 0, 255).astype(np.uint8)

    bg[y1:y2, x1:x2] = blended

    # YOLO bbox
    bbox_x = (x1 + x2) / 2 / bg.shape[1]
    bbox_y = (y1 + y2) / 2 / bg.shape[0]
    bbox_w = (x2 - x1) / bg.shape[1]
    bbox_h = (y2 - y1) / bg.shape[0]

    return bg, (bbox_x, bbox_y, bbox_w, bbox_h)

# --- Main generation logic ---
def generate_augmented_images():
    # Track sign usage by PATH only (memory efficient)
    sign_usage = {cls: {sign_path: 0 for sign_path in sign_paths_by_class[cls]}
                 for cls in ALL_CLASSES}

    for target_class in CLASSES_TO_GENERATE:
        print(f"\nGenerating images for class {target_class}...")
        generated_count = 0
        bg_index = 0

        while generated_count < TARGET_PER_CLASS:
            # Cycle through background images
            bg_filename = bg_images[bg_index % len(bg_images)]
            bg_index += 1

            bg_path = os.path.join(BG_BASE_DIR, bg_filename)

            if bg_filename not in image_info:
                continue

            img_info = image_info[bg_filename]
            image_id = img_info['id']

            if image_id not in image_areas or not image_areas[image_id]:
                continue

            # Load background image
            bg_img = cv2.imread(bg_path)
            if bg_img is None:
                continue

            areas = image_areas[image_id]

            # Select main sign path
            available_signs = [s for s in sign_paths_by_class[target_class]
                             if sign_usage[target_class][s] < MAX_SIGN_REUSE]

            if not available_signs:
                print(f"No available signs for class {target_class}")
                break

            main_sign_path = random.choice(available_signs)

            # LOAD SIGN ON DEMAND (memory efficient)
            main_sign_img = load_and_process_sign(main_sign_path)
            if main_sign_img is None or main_sign_img.size == 0:
                continue

            # Choose area and place sign
            main_area = random.choice(areas)
            center_x, center_y = get_polygon_center(main_area['segmentation'][0])

            main_sign_resized = resize_to_fit_bbox(main_sign_img, main_area['bbox'])
            mask = create_soft_mask(main_sign_resized)

            bg_with_sign, main_bbox = alpha_blend(bg_img.copy(), main_sign_resized, mask, center_x, center_y)

            # YOLO annotations
            yolo_annotations = [f"{target_class} {main_bbox[0]:.6f} {main_bbox[1]:.6f} {main_bbox[2]:.6f} {main_bbox[3]:.6f}"]

            # Add secondary signs if multiple areas
            other_areas = [area for area in areas if area != main_area]
            for area in other_areas:
                other_class = random.choice([c for c in ALL_CLASSES if c != target_class])
                other_sign_path = random.choice(sign_paths_by_class[other_class])

                # LOAD ON DEMAND
                other_sign_img = load_and_process_sign(other_sign_path)
                if other_sign_img is None or other_sign_img.size == 0:
                    continue

                center_x, center_y = get_polygon_center(area['segmentation'][0])
                other_sign_resized = resize_to_fit_bbox(other_sign_img, area['bbox'])
                other_mask = create_soft_mask(other_sign_resized)

                bg_with_sign, other_bbox = alpha_blend(bg_with_sign, other_sign_resized, other_mask, center_x, center_y)
                yolo_annotations.append(f"{other_class} {other_bbox[0]:.6f} {other_bbox[1]:.6f} {other_bbox[2]:.6f} {other_bbox[3]:.6f}")

            # Save results
            output_filename = f"cls{target_class}_bg{Path(bg_filename).stem}_sign{Path(main_sign_path).stem}_{generated_count:04d}"
            cv2.imwrite(os.path.join(OUT_IMG_DIR, f"{output_filename}.jpg"), bg_with_sign)

            with open(os.path.join(OUT_LBL_DIR, f"{output_filename}.txt"), 'w') as f:
                f.write('\n'.join(yolo_annotations))

            # Update usage and free memory
            sign_usage[target_class][main_sign_path] += 1
            generated_count += 1

            if generated_count % 50 == 0:
                print(f"Generated {generated_count}/{TARGET_PER_CLASS}")
                # Explicit memory cleanup
                import gc
                gc.collect()

        print(f"Completed class {target_class}: {generated_count} images")

# Run generation
generate_augmented_images()
print("\nAugmentation complete!")